# Building simulation for NET 04:
## Basic network with diagonal avenue
This notebook summarizes the simulation construction and running for net 04

In [1]:
# Defining network file
NET_FILE = ".\data/grid01-bcn_100.net.xml"

### Demand generation
Endogenous demand based on ACTIVITYGEN

In [2]:
## Config file for parsing demographic data:

ACTIVITYGEN_STAT_FILE = ".\data/activitygen-bcn_100.stat.xml"

## Basic trip output file (without routing) as a resukt of the demand generation based on ACTIVITYGEN.
TRIPS_FROM_ACTIVITYGEN_STAT = ".\data/activitygen-bcn_100.trips.rou.xml"

## Routes output files from ACTIVITYGEN trips
ROUTES_FROM_DUAROUTER_FROM_ACTIVITYGEN = ".\data/activitygen-bcn_100.rou.xml"

In [ ]:
# Calling ACTIVITYGEN tool in SUMO
# Random seed set for reproductibility purposes

!activitygen --net-file {NET_FILE} \
            --stat-file {ACTIVITYGEN_STAT_FILE} \
            --output-file {TRIPS_FROM_ACTIVITYGEN_STAT} \
            --seed 25101987

In [ ]:
# Routing trips from ACTIVITYGEN
# CH routing algorithm is used

!duarouter --net-file {NET_FILE} \
            --route-files {TRIPS_FROM_ACTIVITYGEN_STAT} \
            --output-file {ROUTES_FROM_DUAROUTER_FROM_ACTIVITYGEN} \
            --routing-algorithm CH \
            --ignore-errors

Passing through traffic from random trips between fringe edges

In [ ]:
# creation of hourly distribution of trips
dist = [215, 220, 246, 470, 1000, 2125, 3780, 5522, 6610, 6624, 5662, 4794, 4390, 4548, 4896, 5472, 6182, 6515, 6388, 5295, 3911, 2618, 1669, 1214]
total_random_trips = 280000
factor = total_random_trips/sum(dist)
random_dist = [int(element * factor) for element in dist]

In [ ]:
# Creating iteratively random trips accordingly to the hourly demand
## CREATE RANDOM FOLDER if it does not exist
import os
if not os.path.exists('data/random'):
    os.makedirs('data/random')

for i in range(len(random_dist)):
    begin = 3600*i
    end = 3600*(i+1)
    trips = random_dist[i]
    period = (end-begin)/trips
    trip_prefix = str(str(i) + "h")
    output_folder_1 = "data/random/random_h_"
    output_folder_2_trips = "-bcn_100.trips.rou.xml"
    output_folder_2_routes = "-bcn_100.rou.xml"

    BEGIN_ARG = str("-b " + str(begin))
    END_ARG = str("-e " + str(end))
    PERIOD_ARG = str("-p" + str(period))
    OUT_TRIPS_ARG = str("-o" + output_folder_1 + str(i) + output_folder_2_trips)
    OUT_ROUTES_ARG = str("-r" + output_folder_1 + str(i) + output_folder_2_routes)
    
    # bash SUMO command for RANDOM trips
    # Adjust the installation path for SUMO
    !python "C:\Program Files (x86)\Eclipse\Sumo\tools\randomTrips.py" -n {NET_FILE} \
    --prefix={trip_prefix} \
    --fringe-factor 100 \
    {BEGIN_ARG} \
    {END_ARG} \
    {PERIOD_ARG} \
    --binomial 10 \
    "{OUT_TRIPS_ARG}" \
    --edge-permission=passenger

In [ ]:
# Merging all random routing files i|n a single file
import os
list_trips_files = []
for file in os.listdir(r"data/random/"):
    if file.endswith(".trips.rou.xml"):
        list_trips_files.append(str("data/random/" + file))
list_trips_files_str = ' '.join([str(str(elem) + ',') for elem in list_trips_files])
list_trips_files_str = list_trips_files_str.replace(" ","")

SINGLE_RANDOM_ROUTES_FILE = "data/random/random_ALL.rou.xml"

!duarouter --net-file {NET_FILE} \
--route-files {list_trips_files_str[0:-1]} \
--output-file {SINGLE_RANDOM_ROUTES_FILE} \
--routing-algorithm CH \
--ignore-errors

### Microscopic Simulation
#### Setting configuration files
A XML configuration file (**data/compar_102.sumo.cfg**) is set for taking all the demand inputs, the net, and specifying the output files (in configuration file **data/init_edgedata_additional_file.xml**) for results colletion.

In [3]:
# Run simulation
## A scaling factor is introduced to tests 40 different scenarios with increasing traffic demand.

#create results folder
import os, time
if not os.path.exists('data/results'):
    os.makedirs('data/results')

total_veh_ref = 315000 # number of vehicles for scale factor 1.0
scale = 2

# running a loop from scale factor 2.0 to scale factor 0.05 and writing the results files.
while scale > 0.0:
    start = time.time()
    print('STARTING sumo simulation with scale {:04.2f}'.format(scale))
    scaled_veh_ref = total_veh_ref * scale
    max_num_teleports = int(scaled_veh_ref * 0.005) #value for stopping the simulation at that iteration
    OUTPUT_PREFIX_ARG = '--output-prefix scale_{:04.2f}_'.format(scale)
    STAT_OUT_ARG = '--statistic-output data/results/overall_stats.xml'.format(scale)
    SUMMARY_ARG = '--summary data/results/summary.xml'.format(scale)
    MAX_TELEP_ARG = '--max-num-teleports {:d}'.format(max_num_teleports)
    SCALE_ARG = '--scale {:f}'.format(scale)
    
    # running the simulation
    # ref https://sumo.dlr.de/docs/sumo.html
    !sumo -c .\data/compar_102.sumo.cfg \
    {OUTPUT_PREFIX_ARG} \
    {STAT_OUT_ARG} \
    {SUMMARY_ARG} \
    {MAX_TELEP_ARG} \
    {SCALE_ARG} \
    --threads 1 \
    -W
    
    # measuring time for performance
    end = time.time()
    elapsed_time = (end-start)/60
    
    print('\nDONE sumo simulation with scale {:04.2f} in {:04.1f} minutes \n'.format(scale, elapsed_time))
    
    scale -= 0.05 #updating scale value

STARTING sumo simulation with scale 2.00
Loading configuration ... done.
Step #0.00 (1ms ~= 1000.00*RT, ~1000.00UPS, vehicles TOT 1 ACT 1 BUF 1)                   
Step #100.00 (1ms ~= 1000.00*RT, ~32000.00UPS, vehicles TOT 34 ACT 32 BUF 2)              
Step #200.00 (1ms ~= 1000.00*RT, ~44000.00UPS, vehicles TOT 59 ACT 44 BUF 1)              
Step #300.00 (1ms ~= 1000.00*RT, ~73000.00UPS, vehicles TOT 98 ACT 73 BUF 0)              
Step #400.00 (1ms ~= 1000.00*RT, ~87000.00UPS, vehicles TOT 130 ACT 87 BUF 0)             
Step #500.00 (1ms ~= 1000.00*RT, ~97000.00UPS, vehicles TOT 168 ACT 97 BUF 0)             
Step #600.00 (1ms ~= 1000.00*RT, ~87000.00UPS, vehicles TOT 200 ACT 87 BUF 0)             
Step #700.00 (1ms ~= 1000.00*RT, ~76000.00UPS, vehicles TOT 221 ACT 76 BUF 3)             
Step #800.00 (1ms ~= 1000.00*RT, ~96000.00UPS, vehicles TOT 264 ACT 96 BUF 0)             
Step #900.00 (2ms ~= 500.00*RT, ~48500.00UPS, vehicles TOT 300 ACT 97 BUF 0)              
Step #1000.00 (2m


Step #20400.00 (38ms ~= 26.32*RT, ~63394.74UPS, vehicles TOT 21819 ACT 2409 BUF 4183)     
Step #20500.00 (28ms ~= 35.71*RT, ~87357.14UPS, vehicles TOT 21971 ACT 2446 BUF 4479)     
Step #20600.00 (32ms ~= 31.25*RT, ~77562.50UPS, vehicles TOT 22070 ACT 2482 BUF 4826)     
Step #20700.00 (39ms ~= 25.64*RT, ~63923.08UPS, vehicles TOT 22140 ACT 2493 BUF 5176)     
Step #20800.00 (28ms ~= 35.71*RT, ~90714.29UPS, vehicles TOT 22234 ACT 2540 BUF 5558)     
Step #20900.00 (35ms ~= 28.57*RT, ~72714.29UPS, vehicles TOT 22310 ACT 2545 BUF 5938)     
Step #21000.00 (32ms ~= 31.25*RT, ~79625.00UPS, vehicles TOT 22365 ACT 2548 BUF 6401)     
Step #21100.00 (43ms ~= 23.26*RT, ~60604.65UPS, vehicles TOT 22468 ACT 2606 BUF 6750)     
Step #21200.00 (34ms ~= 29.41*RT, ~78764.71UPS, vehicles TOT 22600 ACT 2678 BUF 7106)     
Step #21300.00 (43ms ~= 23.26*RT, ~63279.07UPS, vehicles TOT 22716 ACT 2721 BUF 7490)     
Step #21400.00 (43ms ~= 23.26*RT, ~63511.63UPS, vehicles TOT 22823 ACT 2731 BUF 7875)    

Loading configuration ... done.
Step #0.00 (0ms ?*RT. ?UPS, vehicles TOT 1 ACT 1 BUF 1)                                   
Step #100.00 (1ms ~= 1000.00*RT, ~31000.00UPS, vehicles TOT 33 ACT 31 BUF 2)              
Step #200.00 (1ms ~= 1000.00*RT, ~41000.00UPS, vehicles TOT 57 ACT 41 BUF 1)              
Step #300.00 (3ms ~= 333.33*RT, ~23333.33UPS, vehicles TOT 94 ACT 70 BUF 0)               
Step #400.00 (5ms ~= 200.00*RT, ~15800.00UPS, vehicles TOT 123 ACT 79 BUF 0)              
Step #500.00 (4ms ~= 250.00*RT, ~23250.00UPS, vehicles TOT 159 ACT 93 BUF 0)              
Step #600.00 (2ms ~= 500.00*RT, ~41000.00UPS, vehicles TOT 190 ACT 82 BUF 0)              
Step #700.00 (3ms ~= 333.33*RT, ~24666.67UPS, vehicles TOT 210 ACT 74 BUF 2)              
Step #800.00 (2ms ~= 500.00*RT, ~46000.00UPS, vehicles TOT 250 ACT 92 BUF 0)              
Step #900.00 (2ms ~= 500.00*RT, ~47500.00UPS, vehicles TOT 285 ACT 95 BUF 0)              
Step #1000.00 (1ms ~= 1000.00*RT, ~94000.00UPS, vehicles T

Loading configuration ... done.
Step #0.00 (0ms ?*RT. ?UPS, vehicles TOT 1 ACT 1 BUF 1)                                   
Step #100.00 (0ms ?*RT. ?UPS, vehicles TOT 32 ACT 30 BUF 2)                               
Step #200.00 (0ms ?*RT. ?UPS, vehicles TOT 55 ACT 39 BUF 1)                               
Step #300.00 (1ms ~= 1000.00*RT, ~66000.00UPS, vehicles TOT 91 ACT 66 BUF 0)              
Step #400.00 (1ms ~= 1000.00*RT, ~73000.00UPS, vehicles TOT 120 ACT 73 BUF 0)             
Step #500.00 (2ms ~= 500.00*RT, ~43500.00UPS, vehicles TOT 155 ACT 87 BUF 0)              
Step #600.00 (1ms ~= 1000.00*RT, ~79000.00UPS, vehicles TOT 185 ACT 79 BUF 0)             
Step #700.00 (1ms ~= 1000.00*RT, ~69000.00UPS, vehicles TOT 204 ACT 69 BUF 3)             
Step #800.00 (1ms ~= 1000.00*RT, ~88000.00UPS, vehicles TOT 244 ACT 88 BUF 0)             
Step #900.00 (2ms ~= 500.00*RT, ~43000.00UPS, vehicles TOT 277 ACT 86 BUF 0)              
Step #1000.00 (1ms ~= 1000.00*RT, ~90000.00UPS, vehicles T

Step #13200.00 (5ms ~= 200.00*RT, ~51800.00UPS, vehicles TOT 5993 ACT 259 BUF 0)          
Step #13300.00 (3ms ~= 333.33*RT, ~79666.67UPS, vehicles TOT 6068 ACT 239 BUF 2)          
Step #13400.00 (4ms ~= 250.00*RT, ~67500.00UPS, vehicles TOT 6172 ACT 270 BUF 0)          
Step #13500.00 (5ms ~= 200.00*RT, ~57200.00UPS, vehicles TOT 6267 ACT 286 BUF 4)          
Step #13600.00 (4ms ~= 250.00*RT, ~69250.00UPS, vehicles TOT 6346 ACT 277 BUF 0)          
Step #13700.00 (3ms ~= 333.33*RT, ~89000.00UPS, vehicles TOT 6450 ACT 267 BUF 5)          
Step #13800.00 (3ms ~= 333.33*RT, ~89666.67UPS, vehicles TOT 6544 ACT 269 BUF 3)          
Step #13900.00 (5ms ~= 200.00*RT, ~56400.00UPS, vehicles TOT 6632 ACT 282 BUF 2)          
Step #14000.00 (5ms ~= 200.00*RT, ~53600.00UPS, vehicles TOT 6720 ACT 268 BUF 1)          
Step #14100.00 (4ms ~= 250.00*RT, ~67500.00UPS, vehicles TOT 6812 ACT 270 BUF 1)          
Step #14200.00 (4ms ~= 250.00*RT, ~67250.00UPS, vehicles TOT 6893 ACT 269 BUF 1)          

...

Step #24000.00 (2ms ~= 500.00*RT, ~91000.00UPS, vehicles TOT 3805 ACT 182 BUF 0)          
Step #24100.00 (1ms ~= 1000.00*RT, ~200000.00UPS, vehicles TOT 3872 ACT 200 BUF 0)        
Step #24200.00 (1ms ~= 1000.00*RT, ~201000.00UPS, vehicles TOT 3933 ACT 201 BUF 0)        
Step #24300.00 (3ms ~= 333.33*RT, ~67000.00UPS, vehicles TOT 3994 ACT 201 BUF 0)          
Step #24400.00 (2ms ~= 500.00*RT, ~98500.00UPS, vehicles TOT 4059 ACT 197 BUF 0)          
Step #24500.00 (1ms ~= 1000.00*RT, ~200000.00UPS, vehicles TOT 4123 ACT 200 BUF 0)        
Step #24600.00 (2ms ~= 500.00*RT, ~97000.00UPS, vehicles TOT 4185 ACT 194 BUF 0)          
Step #24700.00 (1ms ~= 1000.00*RT, ~198000.00UPS, vehicles TOT 4248 ACT 198 BUF 0)        
Step #24800.00 (2ms ~= 500.00*RT, ~105000.00UPS, vehicles TOT 4317 ACT 210 BUF 0)         
Step #24900.00 (3ms ~= 333.33*RT, ~62000.00UPS, vehicles TOT 4376 ACT 186 BUF 0)          
Step #25000.00 (1ms ~= 1000.00*RT, ~177000.00UPS, vehicles TOT 4436 ACT 177 BUF 0)        

Loading configuration ... done.
Step #0.00 (0ms ?*RT. ?UPS, vehicles TOT 1 ACT 1 BUF 0)                                   
Step #100.00 (0ms ?*RT. ?UPS, vehicles TOT 2 ACT 2 BUF 0)                                 
Step #200.00 (0ms ?*RT. ?UPS, vehicles TOT 3 ACT 3 BUF 0)                                 
Step #300.00 (1ms ~= 1000.00*RT, ~5000.00UPS, vehicles TOT 5 ACT 5 BUF 0)                 
Step #400.00 (0ms ?*RT. ?UPS, vehicles TOT 7 ACT 4 BUF 0)                                 
Step #500.00 (0ms ?*RT. ?UPS, vehicles TOT 9 ACT 5 BUF 0)                                 
Step #600.00 (0ms ?*RT. ?UPS, vehicles TOT 10 ACT 4 BUF 0)                                
Step #700.00 (0ms ?*RT. ?UPS, vehicles TOT 11 ACT 4 BUF 0)                                
Step #800.00 (0ms ?*RT. ?UPS, vehicles TOT 13 ACT 3 BUF 0)                                
Step #900.00 (0ms ?*RT. ?UPS, vehicles TOT 15 ACT 4 BUF 0)                                
Step #1000.00 (0ms ?*RT. ?UPS, vehicles TOT 17 ACT 6 BUF 0

Step #28300.00 (2ms ~= 500.00*RT, ~93000.00UPS, vehicles TOT 4889 ACT 186 BUF 0)          
Step #28400.00 (2ms ~= 500.00*RT, ~94500.00UPS, vehicles TOT 4951 ACT 189 BUF 0)          
Step #28500.00 (2ms ~= 500.00*RT, ~90000.00UPS, vehicles TOT 5014 ACT 180 BUF 0)          
Step #28600.00 (1ms ~= 1000.00*RT, ~181000.00UPS, vehicles TOT 5075 ACT 181 BUF 0)        
Step #28700.00 (2ms ~= 500.00*RT, ~91000.00UPS, vehicles TOT 5135 ACT 182 BUF 0)          
Step #28800.00 (3ms ~= 333.33*RT, ~61000.00UPS, vehicles TOT 5196 ACT 183 BUF 0)          
Step #28900.00 (2ms ~= 500.00*RT, ~101500.00UPS, vehicles TOT 5264 ACT 203 BUF 0)         
Step #29000.00 (1ms ~= 1000.00*RT, ~200000.00UPS, vehicles TOT 5333 ACT 200 BUF 0)        
Step #29100.00 (2ms ~= 500.00*RT, ~106500.00UPS, vehicles TOT 5407 ACT 213 BUF 0)         
Step #29200.00 (1ms ~= 1000.00*RT, ~219000.00UPS, vehicles TOT 5478 ACT 219 BUF 0)        
Step #29300.00 (2ms ~= 500.00*RT, ~112000.00UPS, vehicles TOT 5550 ACT 224 BUF 0)         

Loading configuration ... done.
Step #0.00 (0ms ?*RT. ?UPS, vehicles TOT 1 ACT 1 BUF 0)                                   
Step #100.00 (0ms ?*RT. ?UPS, vehicles TOT 1 ACT 1 BUF 0)                                 
Step #200.00 (1ms ~= 1000.00*RT, ~2000.00UPS, vehicles TOT 2 ACT 2 BUF 0)                 
Step #300.00 (0ms ?*RT. ?UPS, vehicles TOT 3 ACT 3 BUF 0)                                 
Step #400.00 (0ms ?*RT. ?UPS, vehicles TOT 4 ACT 3 BUF 0)                                 
Step #500.00 (0ms ?*RT. ?UPS, vehicles TOT 5 ACT 4 BUF 0)                                 
Step #600.00 (0ms ?*RT. ?UPS, vehicles TOT 5 ACT 1 BUF 0)                                 
Step #700.00 (0ms ?*RT. ?UPS, vehicles TOT 6 ACT 2 BUF 0)                                 
Step #800.00 (0ms ?*RT. ?UPS, vehicles TOT 7 ACT 1 BUF 0)                                 
Step #900.00 (0ms ?*RT. ?UPS, vehicles TOT 8 ACT 1 BUF 0)                                 
Step #1000.00 (0ms ?*RT. ?UPS, vehicles TOT 9 ACT 2 BUF 0)


Step #25700.00 (1ms ~= 1000.00*RT, ~94000.00UPS, vehicles TOT 1668 ACT 94 BUF 0)          
Step #25800.00 (1ms ~= 1000.00*RT, ~94000.00UPS, vehicles TOT 1698 ACT 94 BUF 0)          
Step #25900.00 (1ms ~= 1000.00*RT, ~97000.00UPS, vehicles TOT 1728 ACT 97 BUF 0)          
Step #26000.00 (0ms ?*RT. ?UPS, vehicles TOT 1757 ACT 91 BUF 0)                           
Step #26100.00 (1ms ~= 1000.00*RT, ~86000.00UPS, vehicles TOT 1786 ACT 86 BUF 0)          
Step #26200.00 (1ms ~= 1000.00*RT, ~96000.00UPS, vehicles TOT 1814 ACT 96 BUF 0)          
Step #26300.00 (1ms ~= 1000.00*RT, ~93000.00UPS, vehicles TOT 1844 ACT 93 BUF 0)          
Step #26400.00 (2ms ~= 500.00*RT, ~45500.00UPS, vehicles TOT 1875 ACT 91 BUF 0)           
Step #26500.00 (1ms ~= 1000.00*RT, ~92000.00UPS, vehicles TOT 1904 ACT 92 BUF 0)          
Step #26600.00 (1ms ~= 1000.00*RT, ~96000.00UPS, vehicles TOT 1933 ACT 96 BUF 0)          
Step #26700.00 (1ms ~= 1000.00*RT, ~87000.00UPS, vehicles TOT 1963 ACT 87 BUF 0)         

### Mesoscopic Simulations
#### Setting configuration files
A XML configuration file (**data/compar_102_meso.sumo.cfg**) is set for taking all the demand inputs, the net, and specifying the output files (in configuration file **data/init_edgedata_additional_file_meso.xml**) for results collection.

Three different calibrations are finally run for the mesoscopic case, by changing the headway times ($\tau values$) and other mesoscopic parameters as follows

#### Mesoscopic simulation trial01

Param:

$\tau_{ff} = 1$

$\tau_{fj} = 1.2$

$\tau_{jf} = 1.8$

$\tau_{jj} = 1.4$

--meso-tls-penalty = 2

In [ ]:
# Run simulation
## A scaling factor is introduced to tests 40 different scenarios with increasing traffic demand.

#create results folder

RESULTS_FOLDER_MESO_01 = r'data/results_meso_trial01'

import os, time
if not os.path.exists(RESULTS_FOLDER_MESO_01):
    os.makedirs(RESULTS_FOLDER_MESO_01)

total_veh_ref = 315000 # number of vehicles for scale factor 1.0
scale = 2

# running a loop from scale factor 2.0 to scale factor 0.05 and writing the results files.
while scale > 0.0:
    start = time.time()
    print('STARTING sumo simulation with scale {:04.2f}'.format(scale))
    scaled_veh_ref = total_veh_ref * scale
    max_num_teleports = int(scaled_veh_ref * 0.005) #value for stopping the simulation at that iteration
    OUTPUT_PREFIX_ARG = '--output-prefix scale_{:04.2f}_'.format(scale)
    STAT_OUT_ARG = '--statistic-output {}/overall_stats.xml'.format(RESULTS_FOLDER_MESO_01)
    SUMMARY_ARG = '--summary {}/summary.xml'.format(RESULTS_FOLDER_MESO_01)
    MAX_TELEP_ARG = '--max-num-teleports {:d}'.format(max_num_teleports)
    SCALE_ARG = '--scale {:f}'.format(scale)
    
    # running the simulation
    # ref https://sumo.dlr.de/docs/sumo.html
    !sumo -c .\data/compar_102_meso.sumo.cfg \
    --mesosim \
    --meso-edgelength 135 \
    --meso-tauff 1 \
    --meso-taufj 1.2 \
    --meso-taujf 1.8 \
    --meso-taujj 1.4 \
    --meso-lane-queue \
    --meso-tls-penalty 2 \
    --meso-overtaking \
    {OUTPUT_PREFIX_ARG} \
    {STAT_OUT_ARG} \
    {SUMMARY_ARG} \
    {MAX_TELEP_ARG} \
    {SCALE_ARG} \
    --threads 1
    
    # measuring time for performance
    end = time.time()
    elapsed_time = (end-start)/60
    
    print('\nDONE sumo simulation with scale {:04.2f} in {:04.1f} minutes \n'.format(scale, elapsed_time))
    
    scale -= 0.05 #updating scale value

#### Mesoscopic simulation trial02

Param:

$\tau_{ff} = 2.1$

$\tau_{fj} = 0.75$

$\tau_{jf} = 0.75$

$\tau_{jj} = 0.5$

--meso-jam-threshold = -0.5

--meso-junction-control True

In [ ]:
# Run simulation
## A scaling factor is introduced to tests 40 different scenarios with increasing traffic demand.

#create results folder

RESULTS_FOLDER_MESO_02 = r'data/results_meso_trial02'

import os, time
if not os.path.exists(RESULTS_FOLDER_MESO_02):
    os.makedirs(RESULTS_FOLDER_MESO_02)

total_veh_ref = 315000 # number of vehicles for scale factor 1.0
scale = 2

# running a loop from scale factor 2.0 to scale factor 0.05 and writing the results files.
while scale > 0.0:
    start = time.time()
    print('STARTING sumo simulation with scale {:04.2f}'.format(scale))
    scaled_veh_ref = total_veh_ref * scale
    max_num_teleports = int(scaled_veh_ref * 0.005) #value for stopping the simulation at that iteration
    OUTPUT_PREFIX_ARG = '--output-prefix scale_{:04.2f}_'.format(scale)
    STAT_OUT_ARG = '--statistic-output {}/overall_stats.xml'.format(RESULTS_FOLDER_MESO_02)
    SUMMARY_ARG = '--summary {}/summary.xml'.format(RESULTS_FOLDER_MESO_02)
    MAX_TELEP_ARG = '--max-num-teleports {:d}'.format(max_num_teleports)
    SCALE_ARG = '--scale {:f}'.format(scale)
    
    # running the simulation
    # ref https://sumo.dlr.de/docs/sumo.html
    !sumo -c .\data/compar_102_meso.sumo.cfg \
    --mesosim \
    --meso-edgelength 135 \
    --meso-tauff 2.1 \
    --meso-taufj 0.75 \
    --meso-taujf 0.75 \
    --meso-taujj 0.5 \
    --meso-lane-queue \
    --meso-jam-threshold -0.5 \
    --meso-junction-control \
    --meso-overtaking \
    {OUTPUT_PREFIX_ARG} \
    {STAT_OUT_ARG} \
    {SUMMARY_ARG} \
    {MAX_TELEP_ARG} \
    {SCALE_ARG} \
    --threads 1
    
    # measuring time for performance
    end = time.time()
    elapsed_time = (end-start)/60
    
    print('\nDONE sumo simulation with scale {:04.2f} in {:04.1f} minutes \n'.format(scale, elapsed_time))
    
    scale -= 0.05 #updating scale value

#### Mesoscopic simulation trial03

Params:

$\tau_{ff} = 1.13$

$\tau_{fj} = 1.13$

$\tau_{jf} = 1.73$

$\tau_{jj} = 1.4$

--meso-jam-threshold = -1

--meso-junction-control True

In [ ]:
# Run simulation
## A scaling factor is introduced to tests 40 different scenarios with increasing traffic demand.

#create results folder

RESULTS_FOLDER_MESO_03 = r'data/results_meso_trial03'

import os, time
if not os.path.exists(RESULTS_FOLDER_MESO_03):
    os.makedirs(RESULTS_FOLDER_MESO_03)

total_veh_ref = 315000 # number of vehicles for scale factor 1.0
scale = 2

# running a loop from scale factor 2.0 to scale factor 0.05 and writing the results files.
while scale > 0.0:
    start = time.time()
    print('STARTING sumo simulation with scale {:04.2f}'.format(scale))
    scaled_veh_ref = total_veh_ref * scale
    max_num_teleports = int(scaled_veh_ref * 0.005) #value for stopping the simulation at that iteration
    OUTPUT_PREFIX_ARG = '--output-prefix scale_{:04.2f}_'.format(scale)
    STAT_OUT_ARG = '--statistic-output {}/overall_stats.xml'.format(RESULTS_FOLDER_MESO_03)
    SUMMARY_ARG = '--summary {}/summary.xml'.format(RESULTS_FOLDER_MESO_03)
    MAX_TELEP_ARG = '--max-num-teleports {:d}'.format(max_num_teleports)
    SCALE_ARG = '--scale {:f}'.format(scale)
    
    # running the simulation
    # ref https://sumo.dlr.de/docs/sumo.html
    !sumo -c .\data/compar_102_meso.sumo.cfg \
    --mesosim \
    --meso-edgelength 135 \
    --meso-junction-control \
    --meso-lane-queue \
    --meso-overtaking \
    {OUTPUT_PREFIX_ARG} \
    {STAT_OUT_ARG} \
    {SUMMARY_ARG} \
    {MAX_TELEP_ARG} \
    {SCALE_ARG} \
    --threads 1
    
    # measuring time for performance
    end = time.time()
    elapsed_time = (end-start)/60
    
    print('\nDONE sumo simulation with scale {:04.2f} in {:04.1f} minutes \n'.format(scale, elapsed_time))
    
    scale -= 0.05 #updating scale value